<a href="https://colab.research.google.com/github/Putt2020/pytorch_beginner/blob/main/Pytorch_for_beginner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:02<00:00, 9928113.64it/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 201589.80it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:06<00:00, 675389.54it/s] 


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 22283051.59it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [3]:
""" Define dataloader for train and test """
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [4]:
""" define Model """
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
""" Loss function for model """
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [6]:
""" Define train test function """
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
""" Train Model """
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.308218  [   64/60000]
loss: 2.295139  [ 6464/60000]
loss: 2.271226  [12864/60000]
loss: 2.264940  [19264/60000]
loss: 2.252426  [25664/60000]
loss: 2.215834  [32064/60000]
loss: 2.240029  [38464/60000]
loss: 2.201779  [44864/60000]
loss: 2.188384  [51264/60000]
loss: 2.160230  [57664/60000]
Test Error: 
 Accuracy: 33.8%, Avg loss: 2.153940 

Epoch 2
-------------------------------
loss: 2.163889  [   64/60000]
loss: 2.156354  [ 6464/60000]
loss: 2.094474  [12864/60000]
loss: 2.116470  [19264/60000]
loss: 2.064284  [25664/60000]
loss: 1.998441  [32064/60000]
loss: 2.044420  [38464/60000]
loss: 1.960733  [44864/60000]
loss: 1.963473  [51264/60000]
loss: 1.892882  [57664/60000]
Test Error: 
 Accuracy: 55.3%, Avg loss: 1.886419 

Epoch 3
-------------------------------
loss: 1.919971  [   64/60000]
loss: 1.892129  [ 6464/60000]
loss: 1.768477  [12864/60000]
loss: 1.817862  [19264/60000]
loss: 1.698958  [25664/60000]
loss: 1.652034  [32064/600

In [8]:
""" Save Model """
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [9]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


In [ ]:
# from __future__ import print_function
# import keras
# from keras.datasets import mnist
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Flatten
# from keras.layers import Conv2D, MaxPooling2D
# from keras import backend as K

# batch_size = 128
# num_classes = 10
# epochs = 500

# # input image dimensions
# img_rows, img_cols = 28, 28

# # the data, split between train and test sets
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# if K.image_data_format() == 'channels_first':
#     x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
#     x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
#     input_shape = (1, img_rows, img_cols)
# else:
#     x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#     x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
#     input_shape = (img_rows, img_cols, 1)

# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255
# print('x_train shape:', x_train.shape)
# print(x_train.shape[0], 'train samples')
# print(x_test.shape[0], 'test samples')

# # convert class vectors to binary class matrices
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)

# model = Sequential()
# model.add(Conv2D(6, kernel_size=(5, 5),
#                  strides=(1,1),
#                  activation='tanh',
#                  input_shape=input_shape))
# model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2)))
# model.add(Conv2D(6, (5, 5), strides=(1,1),activation='tanh'))
# model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2)))
# # model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(120, activation='tanh'))
# # model.add(Dropout(0.25))
# model.add(Dense(84, activation='tanh'))
# # model.add(Dropout(0.25))
# model.add(Dense(num_classes, activation='softmax'))

# model.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.Adadelta(),
#               metrics=['accuracy'])

# model.fit(x_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_split=0.1)
# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

11490434/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Epoch 1/500
422/422 [==============================] - 12s 10ms/step - loss: 2.3334 - accuracy: 0.0934 - val_loss: 2.3171 - val_accuracy: 0.0910
Epoch 2/500
422/422 [==============================] - 3s 6ms/step - loss: 2.3005 - accuracy: 0.1026 - val_loss: 2.2844 - val_accuracy: 0.1020
Epoch 3/500
422/422 [==============================] - 2s 5ms/step - loss: 2.2700 - accuracy: 0.1182 - val_loss: 2.2532 - val_accuracy: 0.1250
Epoch 4/500
422/422 [==============================] - 2s 4ms/step - loss: 2.2406 - accuracy: 0.1426 - val_loss: 2.2227 - val_accuracy: 0.1543
Epoch 5/500
422/422 [==============================] - 2s 4ms/step - loss: 2.2114 - accuracy: 0.1823 - val_loss: 2.1921 - val_accuracy: 0.2070
Epoch 6/500
422/422 [==============================] - 2s 4ms/step - loss: 2.1818 - accuracy: 0.2331 - val_loss: 2.1607 - val_accuracy: 0.2720
Ep